In [9]:
!pip install sklearn mlflow botocore boto3

     |████████████████████████████████| 131 kB 16.0 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 9.3 MB/s  eta 0:00:01


In [19]:
from prefect import task, Flow, Parameter, Client
from prefect.run_configs import KubernetesRun
from prefect.schedules import IntervalSchedule
from prefect.storage import S3, Local

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

from datetime import timedelta

import numpy as np
import pandas as pd

import mlflow
import requests

In [22]:
#@task
def fetch_data():
    csv_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    data = pd.read_csv(csv_url, sep=";")
    return data

#@task
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

#@task
def train_model(data, mlflow_experiment_id, alpha=0.5, l1_ratio=0.5):
    mlflow.set_tracking_uri("http://mlflow.mlflow:5000")

    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    
    with mlflow.start_run(experiment_id=mlflow_experiment_id):
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

In [23]:
data = fetch_data()
train_model(data, 1)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7493258424553656
  MAE: 0.6128090235079842
  R2: 0.13480608933865978


In [20]:
domain = "mlops.ritza-route53.com"  # the domain where you are hosting Open MLOps
username = "gareth@ritza.co"        # the username you used to register on Open MLOps
password = "DkguP5GsB9yiPk8"        # the password you used to register on Open MLOps
s3_bucket = "another-mlops-bucket"  # the S3 bucket you specified when setting up Open MLOps

prefect_project_name = "Test"         # you can use what you want here
docker_image = "drtools/prefect:wine-classifier-3"    # any docker image that has the required Python dependencies


#auth_url = f"https://{domain}/.ory/kratos/public/self-service/login/api"
prefect_url = f"http://10.110.135.242:4200/graphql"

def create_prefect_flow():
    run_config = KubernetesRun(
        labels=["dev"],
        service_account_name="prefect-server-serviceaccount",
        image=docker_image
    )
    storage = Local('/tmp')

    #session_token = get_prefect_token()
    prefect_client = Client(api_server=prefect_url)
    schedule = IntervalSchedule(interval=timedelta(minutes=2))

    with Flow("train-wine-quality-model", schedule, storage=storage, run_config=run_config) as flow:
        data = fetch_data()
        train_model(data=data, mlflow_experiment_id=1, alpha=0.3, l1_ratio=0.3)

    #prefect_client.create_project(project_name=prefect_project_name)
    training_flow_id = prefect_client.register(flow, project_name=prefect_project_name)
    flow_run_id = prefect_client.create_flow_run(flow_id=training_flow_id, run_name=f"run {prefect_project_name}")
    
create_prefect_flow()

<ipython-input-20-390383d4c8b2>:30: UserWarning: No result handler was specified on your Flow. Cloud features such as input caching and resuming task runs from failure may not work properly.
  training_flow_id = prefect_client.register(flow, project_name=prefect_project_name)


Result check: OK


ClientError: 400 Client Error: Bad Request for url: http://10.110.135.242:4200/graphql

The following error messages were provided by the GraphQL server:

    GRAPHQL_VALIDATION_FAILED: Cannot query field "user" on type "Query".

The GraphQL query was:

    query {
            user {
                default_membership {
                    tenant {
                        slug
                }
            }
        }
    }

The passed variables were:

    null
